In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install catboost

In [ ]:
# !pip install optuna

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import Normalizer
from catboost import CatBoostClassifier
import random
import re
import os


## Define Function

In [2]:
def read_all_file_paths(root_directory='/content/drive/MyDrive/Colab Notebooks/notebook/5002 final/Data_Q5-20231214T121641Z-001/Data_Q5'):
    data_dict = {}
    for root, _, files in os.walk(root_directory):
        for filename in files:
            extracted_text = re.match(r'^([^.]*)\.', filename)
            if extracted_text:
                data_dict[extracted_text.group(1)] = os.path.join(root, filename)
    return data_dict

def preprocess_min_max(data):
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled
    
def preprocess_normalize(data):
    scaler = Normalizer()
    data_scaled = scaler.fit_transform(data)
    return data_scaled
    


## Feature Engineering

In [3]:
def categorize_age(age):
    # Using a single return statement for each age group
    return 1 if 18 <= age <= 30 else (2 if age <= 45 else (3 if age <= 60 else 4))

# Assigns age groups based on age

def categorize_blood_status(systolic_value):
    # Simplified conditional return
    return 1 if systolic_value < 120 else 2

# Determines blood status from systolic blood pressure

def calculate_lifestyle_factors(row):
    # Inline evaluation of conditions
    return sum([
        row['age'] <= 30,  # Younger or equal to 30 years
        row['BMI'] <= 25,  # BMI 25 or less
        row['blood status'] == 1,  # Normal blood status
        row['CholesterolRatio'] <= 4.0,  # Cholesterol ratio up to 4.0
        row['KidneyFunctionIndicator'] <= 0.2  # Adequate kidney function
    ])

In [4]:
# Calculates a lifestyle factor score for each record

# Reading and preprocessing data
data_paths = read_all_file_paths()
data = pd.read_csv(data_paths["train"]).drop(columns=['id'])
data['BMI'] = data['weight(kg)'] / (data['height(cm)'] / 100) ** 2
data['blood status'] = data['systolic'].apply(categorize_blood_status)
data['CholesterolRatio'] = data['HDL'] / data['LDL']
data['KidneyFunctionIndicator'] = data['serum creatinine'] * ((data['AST'] + data['ALT']) / 2)
data['LifestyleFactors'] = data.apply(calculate_lifestyle_factors, axis=1)

## Train

In [5]:
X, y = data.drop(['smoking'],axis=1), data['smoking']
# X = preprocess_min_max(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
#{'random_state': 20, 'learning_rate': 0.1, 'iterations': 1000, 'depth': 6}
param_dist = {
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [6, 8, 10],
    'iterations': [100, 200, 300,1000],
    'random_state':[20,24,42]
    
}
# Create the CatBoostClassifier
catboost = CatBoostClassifier()

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(catboost, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

0:	learn: 0.6397757	total: 149ms	remaining: 14.8s
1:	learn: 0.6011028	total: 236ms	remaining: 11.6s
2:	learn: 0.5711608	total: 307ms	remaining: 9.93s
3:	learn: 0.5484396	total: 379ms	remaining: 9.1s
4:	learn: 0.5308501	total: 454ms	remaining: 8.62s
5:	learn: 0.5167595	total: 524ms	remaining: 8.21s
6:	learn: 0.5052847	total: 596ms	remaining: 7.92s
7:	learn: 0.4968644	total: 666ms	remaining: 7.66s
8:	learn: 0.4896813	total: 734ms	remaining: 7.42s
9:	learn: 0.4834756	total: 804ms	remaining: 7.24s
10:	learn: 0.4782585	total: 874ms	remaining: 7.07s
11:	learn: 0.4738707	total: 944ms	remaining: 6.92s
12:	learn: 0.4698200	total: 1.01s	remaining: 6.78s
13:	learn: 0.4665619	total: 1.08s	remaining: 6.65s
14:	learn: 0.4637441	total: 1.15s	remaining: 6.52s
15:	learn: 0.4614135	total: 1.22s	remaining: 6.42s
16:	learn: 0.4595094	total: 1.29s	remaining: 6.3s
17:	learn: 0.4574058	total: 1.36s	remaining: 6.19s
18:	learn: 0.4556275	total: 1.43s	remaining: 6.09s
19:	learn: 0.4539685	total: 1.5s	remaining:

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

0:	learn: 0.6405350	total: 68.9ms	remaining: 6.83s
1:	learn: 0.6020999	total: 140ms	remaining: 6.88s
2:	learn: 0.5706500	total: 217ms	remaining: 7.02s
3:	learn: 0.5478453	total: 294ms	remaining: 7.05s
4:	learn: 0.5290507	total: 365ms	remaining: 6.93s
5:	learn: 0.5151888	total: 438ms	remaining: 6.87s
6:	learn: 0.5043898	total: 510ms	remaining: 6.77s
7:	learn: 0.4953041	total: 580ms	remaining: 6.67s
8:	learn: 0.4880943	total: 649ms	remaining: 6.56s
9:	learn: 0.4819270	total: 720ms	remaining: 6.48s
10:	learn: 0.4770115	total: 789ms	remaining: 6.38s
11:	learn: 0.4724803	total: 859ms	remaining: 6.3s
12:	learn: 0.4687547	total: 932ms	remaining: 6.23s
13:	learn: 0.4656390	total: 1.03s	remaining: 6.34s
14:	learn: 0.4626388	total: 1.14s	remaining: 6.46s
15:	learn: 0.4601966	total: 1.23s	remaining: 6.48s
16:	learn: 0.4581754	total: 1.32s	remaining: 6.46s
17:	learn: 0.4559230	total: 1.41s	remaining: 6.42s
18:	learn: 0.4540118	total: 1.5s	remaining: 6.38s
19:	learn: 0.4523225	total: 1.58s	remainin

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

...

1:	learn: 0.6830066	total: 42.2ms	remaining: 4.18s
2:	learn: 0.6780859	total: 63.1ms	remaining: 4.14s
3:	learn: 0.6731934	total: 84.1ms	remaining: 4.12s
4:	learn: 0.6686352	total: 104ms	remaining: 4.06s
5:	learn: 0.6641779	total: 128ms	remaining: 4.15s
6:	learn: 0.6594945	total: 150ms	remaining: 4.13s
7:	learn: 0.6552707	total: 169ms	remaining: 4.06s
8:	learn: 0.6511456	total: 189ms	remaining: 4.02s
9:	learn: 0.6471701	total: 209ms	remaining: 3.98s
10:	learn: 0.6431088	total: 228ms	remaining: 3.92s
11:	learn: 0.6391351	total: 251ms	remaining: 3.92s
12:	learn: 0.6353006	total: 271ms	remaining: 3.9s
13:	learn: 0.6315907	total: 292ms	remaining: 3.88s
14:	learn: 0.6278818	total: 316ms	remaining: 3.89s
15:	learn: 0.6242241	total: 336ms	remaining: 3.86s
16:	learn: 0.6209183	total: 356ms	remaining: 3.83s
17:	learn: 0.6176444	total: 376ms	remaining: 3.8s
18:	learn: 0.6144001	total: 396ms	remaining: 3.77s
19:	learn: 0.6113382	total: 418ms	remaining: 3.76s
20:	learn: 0.6082229	total: 440ms	remai

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

0:	learn: 0.6878696	total: 22.2ms	remaining: 4.42s
1:	learn: 0.6828542	total: 43.2ms	remaining: 4.28s
2:	learn: 0.6779487	total: 64ms	remaining: 4.2s
3:	learn: 0.6731713	total: 86.3ms	remaining: 4.23s
4:	learn: 0.6686934	total: 107ms	remaining: 4.16s
5:	learn: 0.6643267	total: 126ms	remaining: 4.07s
6:	learn: 0.6597338	total: 146ms	remaining: 4.03s
7:	learn: 0.6554788	total: 166ms	remaining: 3.97s
8:	learn: 0.6511531	total: 186ms	remaining: 3.96s
9:	learn: 0.6471608	total: 206ms	remaining: 3.92s
10:	learn: 0.6432123	total: 228ms	remaining: 3.92s
11:	learn: 0.6393807	total: 250ms	remaining: 3.91s
12:	learn: 0.6355682	total: 270ms	remaining: 3.88s
13:	learn: 0.6319053	total: 290ms	remaining: 3.85s
14:	learn: 0.6284102	total: 310ms	remaining: 3.83s
15:	learn: 0.6247568	total: 331ms	remaining: 3.81s
16:	learn: 0.6211224	total: 351ms	remaining: 3.78s
17:	learn: 0.6177582	total: 371ms	remaining: 3.75s
18:	learn: 0.6146272	total: 391ms	remaining: 3.73s
19:	learn: 0.6115654	total: 411ms	remain

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

0:	learn: 0.6460546	total: 25.1ms	remaining: 25.1s
1:	learn: 0.6096303	total: 50.7ms	remaining: 25.3s
2:	learn: 0.5804843	total: 75.9ms	remaining: 25.2s
3:	learn: 0.5588498	total: 99.2ms	remaining: 24.7s
4:	learn: 0.5419743	total: 121ms	remaining: 24.2s
5:	learn: 0.5283479	total: 145ms	remaining: 24s
6:	learn: 0.5185759	total: 167ms	remaining: 23.7s
7:	learn: 0.5109366	total: 189ms	remaining: 23.5s
8:	learn: 0.5045272	total: 212ms	remaining: 23.3s
9:	learn: 0.4994423	total: 235ms	remaining: 23.3s
10:	learn: 0.4952082	total: 262ms	remaining: 23.5s
11:	learn: 0.4912899	total: 286ms	remaining: 23.6s
12:	learn: 0.4871952	total: 310ms	remaining: 23.5s
13:	learn: 0.4839990	total: 333ms	remaining: 23.5s
14:	learn: 0.4813717	total: 358ms	remaining: 23.5s
15:	learn: 0.4789134	total: 382ms	remaining: 23.5s
16:	learn: 0.4770103	total: 410ms	remaining: 23.7s
17:	learn: 0.4752374	total: 434ms	remaining: 23.7s
18:	learn: 0.4739046	total: 460ms	remaining: 23.7s
19:	learn: 0.4724905	total: 484ms	remai

In [7]:
from sklearn.metrics import roc_curve, roc_auc_score
test_predictions = best_estimator.predict_proba(X_test)[:, 1]
y_pred_binary = best_estimator.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_binary)
roc_auc 

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

0.8641467787866876

## Test

In [8]:
best_estimator.fit(X,y)


0:	learn: 0.6465261	total: 27.7ms	remaining: 27.7s
1:	learn: 0.6100025	total: 55.7ms	remaining: 27.8s
2:	learn: 0.5813059	total: 81.2ms	remaining: 27s
3:	learn: 0.5587524	total: 108ms	remaining: 26.8s
4:	learn: 0.5432536	total: 139ms	remaining: 27.7s
5:	learn: 0.5299819	total: 169ms	remaining: 28s
6:	learn: 0.5193264	total: 195ms	remaining: 27.6s
7:	learn: 0.5114579	total: 221ms	remaining: 27.4s
8:	learn: 0.5046076	total: 249ms	remaining: 27.4s
9:	learn: 0.4985307	total: 273ms	remaining: 27s
10:	learn: 0.4947053	total: 298ms	remaining: 26.8s
11:	learn: 0.4906854	total: 323ms	remaining: 26.6s
12:	learn: 0.4873148	total: 350ms	remaining: 26.6s
13:	learn: 0.4841989	total: 376ms	remaining: 26.5s
14:	learn: 0.4823284	total: 403ms	remaining: 26.5s
15:	learn: 0.4798341	total: 427ms	remaining: 26.3s
16:	learn: 0.4777057	total: 456ms	remaining: 26.4s
17:	learn: 0.4762238	total: 481ms	remaining: 26.3s
18:	learn: 0.4744029	total: 510ms	remaining: 26.3s
19:	learn: 0.4725893	total: 535ms	remaining:

In [9]:
df_test = pd.read_csv(data_dict["test"])
X_test = df_test.drop(['id'], axis = 1)
X_test['BMI'] = X_test['weight(kg)'] / ((X_test['height(cm)']/100) ** 2)
X_test['blood status'] = X_test['systolic'].apply(categorize_blood_status)
X_test['CholesterolRatio'] = X_test['HDL'] / X_test['LDL']
X_test['KidneyFunctionIndicator'] = X_test['serum creatinine'] * ((X_test['AST'] + X_test['ALT']) / 2)
X_test['LifestyleFactors'] = X_test.apply(calculate_lifestyle_factors, axis=1)



# X['AgeGroup'] = X['age'].apply(categorize_age)
test_predictions =  best_estimator.predict_proba(X_test)[:, 1]
submission = pd.DataFrame({'id': df_test['id'], 'smoking': test_predictions})
submission.to_csv('submission.csv', index=False)

In [10]:
submission.head()

id   smoking
0  159256  0.570273
1  159257  0.399011
2  159258  0.327984
3  159259  0.013952
4  159260  0.628904